In [1]:
# Perform necessary imports
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import time
import os
import glob
import torch
from collections import OrderedDict
from torchvision import transforms, models
import PIL
import cv2

Below we determine the path to load the images to test from. When using collab, we need to mound the Google Drive

In [2]:
# Set this flag to indicate if we are using Google Collab for the training process
using_collab = True
if using_collab:
  # We are going to mount google drive to interface with the dataset
  from google.colab import drive
  drive.mount('/content/drive')
  # Ensure the dataset is uploaded to the next location in your google drive account
  dataset_dir = '/content/drive/My Drive/Colab Notebooks/dataset'
else:
  # We are not using collab. Running in the local computer instead
  # Just give relative path to dataset directory
  dataset_dir = '../dataset'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Below we indicate the name of the classes for which the model was already trained. They mush be in the same order as they were loaded for training

In [3]:
# Name of the classes to load
classes_names = ['BOTTLE_OPENER', 'DINNER_FORK', 'DINNER_KNIFE', 'FISH_SLICE', 'KITCHEN_KNIFE', 'LADLE', 
                 'POTATO_PEELER', 'SPATULA', 'SPOON', 'WHISK']
num_classes = len(classes_names)

Below we configure the architecture of the model that was trained. We need to define this since the saved pt file only includes the value of the trained weights and not necessarily the architecture.

In [4]:
# Define already trained Inception-v3 model for learning transfer
model = models.inception_v3(pretrained=True, aux_logits=False)
# Define custom classifier as FCL
classifier = torch.nn.Sequential(OrderedDict([('fc1', torch.nn.Linear(2048, 1024)),
                                              ('relu1', torch.nn.ReLU()),
                                              ('drop1', torch.nn.Dropout(0.2)),
                                              ('fc2', torch.nn.Linear(1024, 512)),
                                              ('relu2', torch.nn.ReLU()),
                                              ('drop2', torch.nn.Dropout(0.2)),
                                              ('fc3', torch.nn.Linear(512, num_classes)),
                                              ('output', torch.nn.LogSoftmax(dim=1))
                                             ]))
    
# Override classifier in model with our custom FCL
model.fc = classifier
# Disable gradient in the parameters of the model, since we don't want to train it again
for param in model.parameters():
    param.requires_grad = False

# Using an incorrect definition of the model like the one below would cause an error when calling load_state_dict for our .pt saved model
# model = models.resnet18(pretrained=True)
# model.fc = torch.nn.Linear(512, num_classes)

Below we load the weights from the training process.

In [5]:
# Determine device to use (to use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
model.to(device)
print("Using device {}".format(device))

Using device cuda


In [6]:
# Load trained model
model.load_state_dict(torch.load('tl_inceptionv3_model.pt'))

<All keys matched successfully>

In [7]:
def pre_process_image(image):
  """Function to pre-process an image before it can be passed through the model."""
  # Convert from opencv image array to pillow image
  image = PIL.Image.fromarray(image)
  # Define necessary transforms to align with what the model expects at the input
  test_transforms = transforms.Compose([transforms.Resize((310, 310)),
                                      transforms.CenterCrop(299),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
  return test_transforms(image).unsqueeze(0).to(device)

In [8]:
def get_prediction(image):
  tensor = pre_process_image(image)
  logps = model.forward(tensor)
  logsoft_prop, predictions = torch.max(logps, 1)
  predicted_class = classes_names[predictions.item()]
  prob = torch.exp(logsoft_prop).item()
  return prob, predicted_class

In [9]:
# Since we are using our model only for inference, switch to `eval` mode:
model.eval();

In [10]:
fails_count = 0
passes_count = 0
total_fails_count = 0
total_passes_count = 0
total_elapsed = 0
# Determine path to all classes in test directory
classes_dirs = sorted(glob.glob(dataset_dir + '/test/*'))

# Go through each of the classes
for class_dir in classes_dirs:
  fails_count = 0
  passes_count = 0
  class_name = os.path.split(class_dir)[-1]
  print("Running class {}".format(class_name))
  # Go through each of the test images in this class
  for class_img_path in glob.glob(class_dir + "/*"):
    # Load image file
    cv_img = cv2.imread(class_img_path)
    # Start benchmark for evaluation
    start_time = time.time()
    # Perform prediction
    prob_class, pred_class = get_prediction(cv_img)
    # Finish benchmark
    end_time = time.time()
    total_elapsed += (end_time - start_time)

    # Increase counters for pass/fail
    if pred_class == class_name:
      passes_count += 1
    else:
      fails_count += 1

  print("Class passes_count = {}".format(passes_count))
  print("Class fails_count = {}".format(fails_count))
  total_count = passes_count + fails_count
  print("Class accuracy = {}\n".format(100 * passes_count / total_count))
  total_fails_count += fails_count
  total_passes_count += passes_count

print("total_passes_count = {}".format(total_passes_count))
print("total_fails_count = {}".format(total_fails_count))
total_count = total_passes_count + total_fails_count
print("accuracy = {}".format(100 * total_passes_count / total_count))
print("average prediction time [ms] = {}".format(1000 * total_elapsed / total_count))

Running class BOTTLE_OPENER
Class passes_count = 17
Class fails_count = 3
Class accuracy = 85.0

Running class DINNER_FORK
Class passes_count = 18
Class fails_count = 2
Class accuracy = 90.0

Running class DINNER_KNIFE
Class passes_count = 13
Class fails_count = 7
Class accuracy = 65.0

Running class FISH_SLICE
Class passes_count = 14
Class fails_count = 6
Class accuracy = 70.0

Running class KITCHEN_KNIFE
Class passes_count = 20
Class fails_count = 0
Class accuracy = 100.0

Running class LADLE
Class passes_count = 15
Class fails_count = 5
Class accuracy = 75.0

Running class POTATO_PEELER
Class passes_count = 18
Class fails_count = 2
Class accuracy = 90.0

Running class SPATULA
Class passes_count = 15
Class fails_count = 5
Class accuracy = 75.0

Running class SPOON
Class passes_count = 20
Class fails_count = 0
Class accuracy = 100.0

Running class WHISK
Class passes_count = 18
Class fails_count = 2
Class accuracy = 90.0

total_passes_count = 168
total_fails_count = 32
accuracy = 84.0
